# With automated change of scan type and abs values

Keep in mind that max values for all scan types are 1 and min are 0 for FA and GM, and ~-1 for RS.

In [17]:
# Import libraries

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os
import random
from collections import defaultdict
import utils_thresholds
import utils_preproc
import sparsification_utils
import pickle
import time

In [ ]:
# Use only to reload the utils_thresholds import
import importlib
importlib.reload(utils_preproc)

In [ ]:
# Load data

basepath = './'
basepath_data = os.path.join(basepath, 'data/data.npy')
basepath_th = os.path.join(basepath, 'results/thresholded/')
basepath_spar = os.path.join(basepath, 'results/sparsified/')
basepath_plots = os.path.join(basepath, 'results/plots/')
basepath_measures = os.path.join(basepath, 'results/measures/')

data = np.load(basepath_data)
print(f'Data succesfully loaded: {data.shape}')

scan_types = ["FA", "GM", "RS"]

all_nodes = {i for i in range(76)}


In [24]:
# FA: Search around 0.3
FA_thresholds = [0, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
# GM: Search around 0.55
GM_thresholds = [0, 0.5, 0.55, 0.575, 0.6]
# RS: Search between 0.1 and 0.3
RS_thresholds = [0, 0.1, 0.15, 0.2, 0.25, 0.3]

all_thresholds = [FA_thresholds, GM_thresholds, RS_thresholds]
all_thr_for_node = [[0, 0.4, 0.45, 0.5], [0, 0.5, 0.55, 0.575, 0.6], [0, 0.1, 0.15, 0.2, 0.25]]
#all_thr_for_node = [[0], [0, 0.5], [0]]


In [ ]:
scan_type = 2
graph_set = data[:,:,:,scan_type]
print(f'Using {scan_types[scan_type]} scans')

In [ ]:
print("Thresholding graphs...")
thresholded_graphs = utils_thresholds.calculate_thresholded_graphs(graph_set, all_thresholds[scan_type])

### Graph measures

In [ ]:
importlib.reload(utils_thresholds)

In [ ]:
save_path = './'#basepath_plots + "thr_" + scan_types[scan_type] + "_graph_measures_"
original_graph_info = utils_preproc.get_number_of_nodes_and_edges(graph_set)
graph_measures = utils_preproc.plot_graph_metrics(thresholded_graphs, original_graph_info, save_path)

### Node measures

In [31]:
N = 10 # Define how many top nodes you want to consider

save_path = basepath_plots + "thr_" + scan_types[scan_type] + "_node_measures_"


In [43]:
# ("Calculating node measures...")
thresholded_graphs_including_originals = thresholded_graphs#utils_preproc.append_original_graphs_to_threshold(thresholded_graphs, graph_set)
thresholded_graphs_including_originals.keys()
# TODO: ATENCIÓ: això ja no cal fer-ho, elq es pot fer és no guardar els originals quan els guardo com a tal, però està bé guardar totes les mesures igualment

dict_keys([0, 0.1, 0.15, 0.2, 0.25, 0.3])

In [44]:
thresholded_graphs_for_node_metrics = {key: thresholded_graphs_including_originals[key] for key in all_thr_for_node[scan_type]}
thresholded_graphs_for_node_metrics.keys()

dict_keys([0, 0.1, 0.15, 0.2, 0.25])

STILL isn't able to calculate the measures for RS scans for the original graphs.

Tant pels originals tal qual com per un cop thresholded amb thr = 0.

In [56]:
for adj_mat in graph_set:
    G = utils_preproc.adjacency_matrix_to_graph(adj_mat)
    measures = utils_preproc.calculate_node_measures(G)
    print('done')

PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')

In [45]:
for thr in thresholded_graphs_for_node_metrics.keys():
    print(thr)
    for graph_idx in range(len(thresholded_graphs_for_node_metrics[thr])):
        measures = utils_preproc.calculate_node_measures(thresholded_graphs_for_node_metrics[thr][graph_idx])
        #print(thr, graph_idx, measures)

0


PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')

In [52]:
node_measures = utils_preproc.calculate_node_measures_per_graph(thresholded_graphs_for_node_metrics)

PowerIterationFailedConvergence: (PowerIterationFailedConvergence(...), 'power iteration failed to converge within 100 iterations')

In [ ]:

    print("Averaging...")
    avg_node_measures = utils_preproc.calculate_average_node_measures_per_threshold(node_measures)
    common_nodes = utils_preproc.find_common_nodes_across_metrics(avg_node_measures, N)
    all_common_nodes = set.intersection(*common_nodes.values())

    print("Plotting!")
    utils_preproc.plot_node_metrics_per_threshold(all_nodes, all_common_nodes, avg_node_measures, save_path)

    # End time
    end_time = time.time()
    # Calculate the elapsed time
    elapsed_time = end_time - start_time
    # Print the elapsed time
    if elapsed_time > 60:
        print(f"Script took {elapsed_time/60:.2f} minutes to run.")
    else:
        print(f"Script took {elapsed_time:.2f} seconds to run.")

In [ ]:
data

## WAIT!!

Clar, hi ha grafs amb valors negatius als edges però ens interessa el valor absolut!!!!!

He borrat els outputs sense voler però estan guardats a la backup '20241023 thr_main_notebook.ipynb'.

# END